In [1]:
import pandas as pd
from thefuzz import process
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
import numpy as np

# Load the first CSV file into a DataFrame
fifa_players = pd.read_csv('male_players.csv')

combined_player_stats = pd.read_csv("filtered_combined_player_stats.csv")

/tmp/ipykernel_2871835/3486642863.py:13: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa_players = pd.read_csv('male_players.csv')


In [2]:
#league_id 
#LaLiga - 53
#Prem - 13
#Bundesliga - 19
#Serie A - 31
#Ligue 1 - 16

filtered_fifa_players = fifa_players[fifa_players['league_id'].isin([53, 13, 19, 31, 16])]

fifa23_players = filtered_fifa_players[filtered_fifa_players['fifa_version'].isin([23])]
fifa23_players.head()
print(fifa23_players.shape[0])

2880


In [3]:
known_names = fifa23_players['long_name']

# Add a new column called 'rating' to the player_stats DataFrame
combined_player_stats['rating'] = None

# Loop through each corrupted name in the player_stats DataFrame
for idx, name in enumerate(combined_player_stats['Player']):
    # Find the best match from the list of known names
    best_match = process.extractOne(name, known_names)
    if best_match is not None:
        matched_name = best_match[0]
        rating = fifa23_players.loc[fifa23_players['long_name'] == matched_name, 'overall'].iloc[0]
        combined_player_stats.at[idx, 'rating'] = rating
        # Additional checks
        squad = combined_player_stats.at[idx, 'Squad']
        born_year = combined_player_stats.at[idx, 'Born']
        # Find the matched player's info
        matched_player = fifa23_players[fifa23_players['long_name'] == matched_name]
        if not matched_player.empty:
            matched_squad = matched_player['club_name'].iloc[0]
            matched_born_year = int(matched_player['dob'].iloc[0].split('-')[0])
            if born_year == matched_born_year:
                combined_player_stats.at[idx, 'rating'] = rating/99
                #print(best_match)
                #print(f"Matched name: {name}, Best match: {matched_name}, Rating: {rating}")
                #print(f"Squad and birth year matched for {name}")
            else:
                combined_player_stats.drop(idx, inplace=True)

In [4]:
# Replace NaN values with 0
combined_player_stats.fillna(0, inplace=True)
#shuffle the players
combined_player_stats = combined_player_stats.sample(frac=1).reset_index(drop=True)

# Split positions into a list
combined_player_stats['Positions_list'] = combined_player_stats['Pos'].str.split(',')

# Get unique positions
unique_positions = set(position for sublist in combined_player_stats['Positions_list'] for position in sublist)

# Create binary indicator variables for each position
for position in unique_positions:
    combined_player_stats[position] = combined_player_stats['Positions_list'].apply(lambda x: 1 if position in x else 0)

# Drop unnecessary columns
combined_player_stats.drop(['Positions_list'], axis=1, inplace=True)

combined_player_stats.head()

# Reorder the columns to make 'rating' the last column
columns_ordered = [col for col in combined_player_stats.columns if col != 'rating'] + ['rating']

# Reassign the DataFrame with the reordered columns
combined_player_stats = combined_player_stats[columns_ordered]

# Display the updated DataFrame
#combined_player_stats.head()

In [12]:
#Linear regression with categorical variables

combined_player_stats['Squad'] = combined_player_stats['Squad'].astype(str)
combined_player_stats['Comp'] = combined_player_stats['Comp'].astype(str)
combined_player_stats['Nation'] = combined_player_stats['Nation'].astype(str)
#combined_player_stats['Pos'] = combined_player_stats['Pos'].astype(str)

# Splitting the data into input (X) and output (Y) variables
X = combined_player_stats.iloc[1:, :-1].drop(columns=['Player','Pos'])
Y = combined_player_stats.iloc[1:, -1]   # Only the last column, excluding first row

# Define categorical columns
categorical_cols = ['Squad', 'Comp', 'Nation']

column_transformer = ColumnTransformer(
    [('onehotencoder', OneHotEncoder(handle_unknown='ignore'), categorical_cols)],
    remainder='passthrough'
)

# Define the linear regression model
model = LinearRegression()

# Create a pipeline
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('regressor', model)
])

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Fit the model
pipeline.fit(X_train, Y_train)

# Predictions
Y_pred = pipeline.predict(X_test)

# Compute evaluation metrics
mae = mean_absolute_error(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(Y_test, Y_pred)

# Print evaluation metrics
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)

Mean Absolute Error (MAE): 0.030769575384558923
Mean Squared Error (MSE): 0.0016752951101287707
Root Mean Squared Error (RMSE): 0.0409303690446198
R-squared (R2): 0.6738410573952047


In [13]:
#Linear regression without categorical vars

X = combined_player_stats.iloc[1:, 5:-1].values
Y = combined_player_stats.iloc[1:, -1]   # Only the last column, excluding first row

# Define the linear regression model
model = LinearRegression()

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Fit the model
model.fit(X_train, Y_train)

# Predictions
Y_pred = model.predict(X_test)

# Compute evaluation metrics
mae = mean_absolute_error(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(Y_test, Y_pred)

# Print evaluation metrics
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)

Mean Absolute Error (MAE): 0.03341796291947891
Mean Squared Error (MSE): 0.0019000173866472244
Root Mean Squared Error (RMSE): 0.04358918887347211
R-squared (R2): 0.6300904491317044


In [14]:
#Linear Regression on just goalkeepers since their metrics are so different to the rest
#of the players, smaller dataset though

In [15]:
#creating GK dataframe 
# Create a new DataFrame containing only players with position 'GK'
gk_players_df = combined_player_stats[combined_player_stats['Pos'] == 'GK'].copy()

# Columns to keep: columns ending with "_GK" and additional columns
cols_to_keep = [col for col in gk_players_df.columns if col.endswith('_GK')] + ['Age', 'Born', 'MP', 'Minutes_Played', 'Mn_per_MP', 'Mins_Per_90', 'Starts', 'PPM_Team.Success', 'onG_Team.Success', 'onGA_Team.Success', 'CrdY', 'CrdR', 'PrgP','rating']

# Create a new DataFrame containing the necessary columns for the GK model
gk_model_df = gk_players_df[cols_to_keep].copy()

print("Number of rows in df:", gk_model_df.shape[0])

Number of rows in df: 296


In [16]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
#Testing goalkeepers on their own
# Remove the first row from the DataFrame to exclude headers or invalid data
gk_model_df = gk_model_df.iloc[1:]

# Split the data into features (X) and target variable (Y)
X = gk_model_df.iloc[:, 5:-1].values  # Excluding 'rating' and other non-feature columns
Y = gk_model_df['rating'].values

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the Ridge regression model
ridge = Ridge()

# Define the hyperparameters grid
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100] 
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error', verbose=1)

# Fit GridSearchCV to find the best hyperparameters
grid_search.fit(X_train, Y_train)

# Get the best model and its hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Predictions
Y_pred = best_model.predict(X_test)

# Compute evaluation metrics
mae = mean_absolute_error(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(Y_test, Y_pred)

# Print best hyperparameters and evaluation metrics
print("Best hyperparameters:", best_params)
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)


Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best hyperparameters: {'alpha': 100}
Mean Absolute Error (MAE): 0.0414436848084614
Mean Squared Error (MSE): 0.002845500647700676
Root Mean Squared Error (RMSE): 0.05334323431983363
R-squared (R2): 0.6128601187439746


In [17]:
# Remove the first row from the DataFrame to exclude headers or invalid data
from sklearn.model_selection import KFold

gk_model_df = gk_model_df.iloc[1:]

# Split the data into features (X) and target variable (Y)
X = gk_model_df.iloc[:, 5:-1].values  # Excluding 'rating' and other categorical columns
Y = gk_model_df['rating'].values

# Define number of splits for k-fold cross-validation
n_splits = 10

# Initialize k-fold cross-validation
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics for each fold
mae_scores, mse_scores, rmse_scores, r2_scores = [], [], [], []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X):
    # Split the data into training and testing sets for this fold
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    # Define the linear regression model
    linear_reg = Ridge(alpha=10)
    
    # Fit the model
    linear_reg.fit(X_train, Y_train)
    
    # Predictions
    Y_pred = linear_reg.predict(X_test)
    
    # Compute evaluation metrics for this fold
    mae = mean_absolute_error(Y_test, Y_pred)
    mse = mean_squared_error(Y_test, Y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(Y_test, Y_pred)
    
    # Append evaluation metrics to the respective lists
    mae_scores.append(mae)
    mse_scores.append(mse)
    rmse_scores.append(rmse)
    r2_scores.append(r2)

# Compute mean evaluation metrics across all folds
mean_mae = np.mean(mae_scores)
mean_mse = np.mean(mse_scores)
mean_rmse = np.mean(rmse_scores)
mean_r2 = np.mean(r2_scores)

# Print mean evaluation metrics
print("Mean Absolute Error (MAE):", mean_mae)
print("Mean Squared Error (MSE):", mean_mse)
print("Root Mean Squared Error (RMSE):", mean_rmse)
print("Mean R-squared (R2):", mean_r2)

Mean Absolute Error (MAE): 0.04273171806218458
Mean Squared Error (MSE): 0.002945195665476465
Root Mean Squared Error (RMSE): 0.05340388626019442
Mean R-squared (R2): 0.5800373073998699
